In [1]:
import numpy as np
import torch

In [2]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)

w = torch.tensor(1.0,requires_grad = True)

# Forward pass

## compute output

In [3]:
y_hat = w*x

In [4]:
print("Y pred",y_hat)

Y pred tensor(1., grad_fn=<MulBackward0>)


## compute loss

In [5]:
loss = (y_hat - y)**2

In [6]:
print(loss)

tensor(1., grad_fn=<PowBackward0>)


# Backward pass

## compute gradients

In [7]:
loss.backward()

In [8]:
print("initial gradient",w.grad)

initial gradient tensor(-2.)


### manual way of gradient descent

In [9]:
#lets assume y = 2*x is the function so model has to learn that the wt is 2 from its trainings

#inputs and outputs
x = np.array([1.0, 2.0, 3.0, 4.0],dtype = 'float32')
y = np.array([2.0, 4.0, 6.0, 8.0],dtype = 'float32')

#weights
w = 0.0

#learning_rate
learning_rate = 0.01

#predictions in forward pass
def forward(x):
    
    y_pred = w*x
    
    return y_pred

#Compute loss
def loss(y_pred,y):
    
    loss = ((y_pred - y)**2).sum()  
    
    return loss

#Loss = (w.X - Y)**2
#Loss = (Y_pred - Y)**2

#Compute gradients
#DL/DW = 1/N * 2.X * (w.X - Y)

def gradient(x,y,y_pred):
    
    grad = np.dot((2*x),(y_pred - y)).mean()
    
    return grad


no_of_epochs = 10
print("Prediction before running epochs",forward(5))

for i in range(no_of_epochs):
    
    #forwardpass
    y_pred = forward(x)
    
    #compute loss
    
    loss2 = loss(y_pred,y)
    
    #compute gradients
    dw = gradient(x,y,y_pred)
        
    w-= learning_rate*dw
    print(f"epoch {i+1} W {w:.3f} Loss {loss2:.3f}" ) 
    
print(f"Prediction after running {no_of_epochs} epochs = ",forward(5))

Prediction before running epochs 0.0
epoch 1 W 1.200 Loss 120.000
epoch 2 W 1.680 Loss 19.200
epoch 3 W 1.872 Loss 3.072
epoch 4 W 1.949 Loss 0.492
epoch 5 W 1.980 Loss 0.079
epoch 6 W 1.992 Loss 0.013
epoch 7 W 1.997 Loss 0.002
epoch 8 W 1.999 Loss 0.000
epoch 9 W 1.999 Loss 0.000
epoch 10 W 2.000 Loss 0.000
Prediction after running 10 epochs =  9.998951268196105


### Autograd

In [10]:
#lets assume y = 2*x is the function so model has to learn that the wt is 2 from its trainings

#inputs and outputs
x = torch.tensor([[1.0],[ 2.0], [3.0], [4.0]],dtype = torch.float32)
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]],dtype = torch.float32)

#weights
w = torch.tensor(0.0,dtype=torch.float32, requires_grad=True)

#learning_rate
learning_rate = 0.01

#predictions in forward pass
def forward(x):
    
    y_pred = w*x
    
    return y_pred

no_of_epochs = 10
print("Prediction before running epochs",forward(5))

for i in range(no_of_epochs):
    
    #forwardpass
    y_pred = forward(x)
    
    #compute loss
    
    loss2 = loss(y_pred,y)
    
    #compute gradients
    loss2.backward()  #calculates the gradient of loss wrt w dl/dW for backpropogation
        
    #this should not be part of computational graph
    with torch.no_grad():
        w-= learning_rate*w.grad
    
    #zero out the gradient
    #else whenever we call bcakward it will accumulte the gradient in w.grad attribute
    w.grad.zero_()
    
    print(f"epoch {i+1} W {w:.3f} Loss {loss2:.3f}" ) 
    
print(f"Prediction after running {no_of_epochs} epochs = ",forward(5))

Prediction before running epochs tensor(0., grad_fn=<MulBackward0>)
epoch 1 W 1.200 Loss 120.000
epoch 2 W 1.680 Loss 19.200
epoch 3 W 1.872 Loss 3.072
epoch 4 W 1.949 Loss 0.492
epoch 5 W 1.980 Loss 0.079
epoch 6 W 1.992 Loss 0.013
epoch 7 W 1.997 Loss 0.002
epoch 8 W 1.999 Loss 0.000
epoch 9 W 1.999 Loss 0.000
epoch 10 W 2.000 Loss 0.000
Prediction after running 10 epochs =  tensor(9.9990, grad_fn=<MulBackward0>)


## Pytorch loss and Pytorch Optimiser

In [11]:
#lets assume y = 2*x is the function so model has to learn that the wt is 2 from its trainings

import torch.nn as nn
from torch.optim import Adam,SGD
#inputs and outputs
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]],dtype = torch.float32)
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]],dtype = torch.float32)

#weights
w = torch.tensor(0.0,dtype=torch.float32, requires_grad=True)

#learning_rate
learning_rate = 0.01

#Pytorch loss and optimiser

loss = nn.MSELoss()

opt = SGD(params=[w], lr = learning_rate)

#predictions in forward pass
def forward(x):
    
    y_pred = w*x
    
    return y_pred

no_of_epochs = 100
print("Prediction before running epochs",forward(5).item())

for i in range(no_of_epochs):
    
    #forwardpass
    y_pred = forward(x)
    
    #compute loss
    
    l = loss(y_pred,y)
    
    #compute gradients
    l.backward()  #calculates the gradient of loss wrt w dl/dW for backpropogation
        
    #update the model parameters or weights
    
    opt.step()
    
    #zero out the gradient
    #else whenever we call bcakward it will accumulte the gradient in w.grad attribute
    opt.zero_grad()
    
    if (i%10 == 0):
        print(f"epoch {i+1} W {w:.3f} Loss {l:.3f}" ) 
    
print(f"Prediction after running {no_of_epochs} epochs = ",forward(5).item())

Prediction before running epochs 0.0
epoch 1 W 0.300 Loss 30.000
epoch 11 W 1.665 Loss 1.163
epoch 21 W 1.934 Loss 0.045
epoch 31 W 1.987 Loss 0.002
epoch 41 W 1.997 Loss 0.000
epoch 51 W 1.999 Loss 0.000
epoch 61 W 2.000 Loss 0.000
epoch 71 W 2.000 Loss 0.000
epoch 81 W 2.000 Loss 0.000
epoch 91 W 2.000 Loss 0.000
Prediction after running 100 epochs =  9.999998092651367


## forward pass using pytorc model

In [13]:
#lets assume y = 2*x is the function so model has to learn that the wt is 2 from its trainings

import torch.nn as nn
from torch.optim import Adam,SGD
#inputs and outputs
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]],dtype = torch.float32)
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]],dtype = torch.float32)

x_test = torch.tensor([5.0],dtype = torch.float32)

n_samples , n_features = x.shape
#learning_rate
learning_rate = 0.01

#Pytorch loss and optimiser

loss = nn.MSELoss()



#predictions in forward pass
model = nn.Linear(in_features = n_features ,out_features =n_features)
opt = SGD(params=model.parameters(), lr = learning_rate)

no_of_epochs = 100
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

for i in range(no_of_epochs):
    
    #forwardpass
    y_pred = model(x)
    
    #compute loss
    
    l = loss(y_pred,y)
    
    #compute gradients
    l.backward()  #calculates the gradient of loss wrt w dl/dW for backpropogation
        
    #update the model parameters or weights
    
    opt.step()
    
    #zero out the gradient
    #else whenever we call bcakward it will accumulte the gradient in w.grad attribute
    opt.zero_grad()
    
    if (i%10 == 0):
        [w, b] = model.parameters() # unpack parameters
        print('epoch ', i+1, ': w = ', w[0][0].item(), ' loss = ', l)
        
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

Prediction before training: f(5) = 1.696
epoch  1 : w =  0.4925873875617981  loss =  tensor(19.3001, grad_fn=<MseLossBackward0>)
epoch  11 : w =  1.5042155981063843  loss =  tensor(0.6296, grad_fn=<MseLossBackward0>)
epoch  21 : w =  1.6744176149368286  loss =  tensor(0.1389, grad_fn=<MseLossBackward0>)
epoch  31 : w =  1.70905601978302  loss =  tensor(0.1191, grad_fn=<MseLossBackward0>)
epoch  41 : w =  1.721674919128418  loss =  tensor(0.1119, grad_fn=<MseLossBackward0>)
epoch  51 : w =  1.7305437326431274  loss =  tensor(0.1054, grad_fn=<MseLossBackward0>)
epoch  61 : w =  1.738607406616211  loss =  tensor(0.0992, grad_fn=<MseLossBackward0>)
epoch  71 : w =  1.7463452816009521  loss =  tensor(0.0934, grad_fn=<MseLossBackward0>)
epoch  81 : w =  1.753840684890747  loss =  tensor(0.0880, grad_fn=<MseLossBackward0>)
epoch  91 : w =  1.7611124515533447  loss =  tensor(0.0829, grad_fn=<MseLossBackward0>)
Prediction before training: f(5) = 9.521


In [16]:
#lets assume y = 2*x is the function so model has to learn that the wt is 2 from its trainings

import torch.nn as nn
from torch.optim import Adam,SGD
#inputs and outputs
x = torch.tensor([[1.0], [2.0], [3.0], [4.0]],dtype = torch.float32)
y = torch.tensor([[2.0], [4.0], [6.0], [8.0]],dtype = torch.float32)

x_test = torch.tensor([5.0],dtype = torch.float32)

n_samples , n_features = x.shape
#learning_rate
learning_rate = 0.01

#Pytorch loss and optimiser

loss = nn.MSELoss()



#predictions in forward pass
class LinearRegression(nn.Module):
    
    def __init__(self,inputs,outputs):
        super(LinearRegression, self).__init__()
        
        self.in_features = inputs
        self.out_features = outputs
        self.ln = nn.Linear(self.in_features ,self.out_features)
        
    def forward(self,x):
        x = self.ln(x)
        return x


model = LinearRegression(n_features,n_features)
opt = SGD(params=model.parameters(), lr = learning_rate)

no_of_epochs = 100
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

for i in range(no_of_epochs):
    
    #forwardpass
    y_pred = model(x)
    
    #compute loss
    
    l = loss(y_pred,y)
    
    #compute gradients
    l.backward()  #calculates the gradient of loss wrt w dl/dW for backpropogation
        
    #update the model parameters or weights
    
    opt.step()
    
    #zero out the gradient
    #else whenever we call bcakward it will accumulte the gradient in w.grad attribute
    opt.zero_grad()
    
    if (i%10 == 0):
        [w, b] = model.parameters() # unpack parameters
        print('epoch ', i+1, ': w = ', w[0][0].item(), ' loss = ', l)
        
print(f'Prediction before training: f(5) = {model(x_test).item():.3f}')

Prediction before training: f(5) = 4.663
epoch  1 : w =  1.0905404090881348  loss =  tensor(8.5253, grad_fn=<MseLossBackward0>)
epoch  11 : w =  1.7618355751037598  loss =  tensor(0.2377, grad_fn=<MseLossBackward0>)
epoch  21 : w =  1.87252676486969  loss =  tensor(0.0223, grad_fn=<MseLossBackward0>)
epoch  31 : w =  1.8929650783538818  loss =  tensor(0.0158, grad_fn=<MseLossBackward0>)
epoch  41 : w =  1.8988085985183716  loss =  tensor(0.0147, grad_fn=<MseLossBackward0>)
epoch  51 : w =  1.9022289514541626  loss =  tensor(0.0139, grad_fn=<MseLossBackward0>)
epoch  61 : w =  1.9051865339279175  loss =  tensor(0.0131, grad_fn=<MseLossBackward0>)
epoch  71 : w =  1.9079984426498413  loss =  tensor(0.0123, grad_fn=<MseLossBackward0>)
epoch  81 : w =  1.9107178449630737  loss =  tensor(0.0116, grad_fn=<MseLossBackward0>)
epoch  91 : w =  1.9133553504943848  loss =  tensor(0.0109, grad_fn=<MseLossBackward0>)
Prediction before training: f(5) = 9.826
